In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import pickle
import re
import math
import random

In [2]:
model_name = "DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda(0)

premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was not good."

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].cuda(0))
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

{'entailment': 83.6, 'neutral': 16.2, 'contradiction': 0.3}


In [3]:
# Function to get a textual entailment prediction for a premise and hypothesis

def get_prediction(premise,hypothesis):
    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].cuda(0))
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    #label_names = ["entailment", "neutral", "contradiction"]
    label_names = ["entailment", "neutral", "contradiction"]
    prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
    return prediction

In [4]:
# Verify all these outputs work as expected.

premise="I really admire you."
hypothesis="I find you despicable."
print(get_prediction(premise,hypothesis))
premise="I really admire you."
hypothesis="You are a person that I look up to."
print(get_prediction(premise,hypothesis))
premise="I really admire you."
hypothesis="I love the outdoors."
print(get_prediction(premise,hypothesis))

{'entailment': 0.0, 'neutral': 0.0, 'contradiction': 100.0}
{'entailment': 99.2, 'neutral': 0.7, 'contradiction': 0.0}
{'entailment': 0.1, 'neutral': 99.9, 'contradiction': 0.0}


In [ ]:
#load pickle binary file

objects = []
with (open("text_list.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
texts=objects[0]